# Lidando com dados duplicados, ausentes ou inválidos

## Sobre os dados
Neste notebook, usaremos dados meteorológicos diários retirados da [API do National Centers for Environmental Information (NCEI)](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) e alterados para introduzir muitos problemas comuns enfrentados ao trabalhar com dados.

*Nota: O NCEI faz parte da National Oceanic and Atmospheric Administration (NOAA) e, como você pode ver pela URL da API, este recurso foi criado quando o NCEI era chamado de NCDC. Caso a URL deste recurso mude no futuro, você pode procurar por "API meteorológica NCEI" para encontrar a nova URL.*

## Contexto sobre os dados

Significados dos dados:
- `PRCP`: precipitação em milímetros
- `SNOW`: queda de neve em milímetros
- `SNWD`: profundidade da neve em milímetros
- `TMAX`: temperatura máxima diária em Celsius
- `TMIN`: temperatura mínima diária em Celsius
- `TOBS`: temperatura no momento da observação em Celsius
- `WESF`: equivalente em água da neve em milímetros

Alguns fatos importantes para nos situarmos:
- Segundo o Serviço Nacional de Meteorologia dos EUA, a temperatura mais baixa já registrada no Central Park foi de -15°F (-26,1°C) em 9 de fevereiro de 1934: [fonte](https://www.weather.gov/media/okx/Climate/CentralPark/extremes.pdf)
- A temperatura da fotosfera do Sol é aproximadamente 5.505°C: [fonte](https://en.wikipedia.org/wiki/Sun)

## Configuração
Precisamos importar o `pandas` e ler os dados sujos para começar:

In [1]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

## Encontrando dados problemáticos
Um bom primeiro passo é olhar algumas linhas:

In [2]:
df.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


Olhar as estatísticas resumidas pode revelar valores estranhos ou ausentes:

In [3]:
df.describe()

/Users/goliveira/Library/Caches/pypoetry/virtualenvs/hands-on-data-analysis-with-pandas-2nd-edi-BuKKwh-Y-py3.10/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/goliveira/Library/Caches/pypoetry/virtualenvs/hands-on-data-analysis-with-pandas-2nd-edi-BuKKwh-Y-py3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


O método `info()` pode identificar valores ausentes e tipos de dados incorretos:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


Podemos usar o método `isna()`/`isnull()` da série para encontrar nulos:

In [7]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0]

765

In [8]:
contain_nulls.head(10)

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


Observe que não podemos verificar se temos `NaN` desta forma:

In [9]:
df[df.inclement_weather == 'NaN'].shape[0]

0

Isso ocorre porque na verdade é `np.nan`. No entanto, observe que isso também não funciona:

In [12]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

0

Precisamos usar um dos métodos discutidos anteriormente para que isso funcione:

In [13]:
df[df.inclement_weather.isna()].shape[0]

357

Podemos encontrar `-inf`/`inf` comparando com `-np.inf`/`np.inf`:

In [14]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

577

Em vez de fazer isso para cada coluna, podemos escrever uma função que utilizará uma [compreensão de dicionário](https://www.python.org/dev/peps/pep-0274/) para verificar todas as colunas para nós:

In [15]:
def get_inf_count(df):
    """Find the number of inf/-inf values per column in the dataframe"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }


get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

Antes de decidirmos como lidar com os valores infinitos da profundidade da neve, devemos olhar as estatísticas resumidas para a queda de neve, que desempenha um papel importante na determinação da profundidade da neve:

In [16]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

,count,mean,std,min,25%,50%,75%,max
np.inf Snow Depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf Snow Depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


Vamos agora investigar as colunas `date` e `station`. Vimos o `?` para a estação anteriormente, então sabemos que era o outro valor único. No entanto, vemos que algumas datas estão presentes 8 vezes nos dados e temos apenas 324 dias, o que significa que também estamos perdendo dias:

In [17]:
df.describe(include='object')

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


Podemos usar o método `duplicated()` para encontrar linhas duplicadas:

In [18]:
df[df.duplicated()].shape[0]

284

Também podemos especificar as colunas a serem usadas:

In [22]:
df[df.duplicated(['date', 'station'])].shape[0]

284

Vamos olhar para alguns duplicados.

In [23]:
df[df.duplicated()].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


## Mitigando Problemas

### Lidando com dados duplicados
Como sabemos que temos dados meteorológicos de NY e notamos que temos apenas duas entradas para a coluna `station`, podemos decidir descartar a coluna `station` porque estamos interessados apenas nos dados meteorológicos. No entanto, ao lidar com dados duplicados, precisamos pensar nas ramificações de removê-los. Observe que só temos dados para a coluna `WESF` quando a estação é `?`:

In [25]:
df[df.WESF.notna()].station.unique()

array(['?'], dtype=object)

Se determinarmos que isso não afetará nossa análise, podemos usar `drop_duplicates()` para removê-los:

In [26]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates(
    'date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.shape

(324, 8)

Aqui usamos o método `combine_first()` para combinar os valores para a primeira entrada não nula; isso significa que, se tivéssemos dados de ambas as estações, primeiro pegaríamos o valor fornecido pela estação nomeada e, apenas se essa estação fosse nula, pegaríamos o valor da estação nomeada `?`. A tabela a seguir contém alguns exemplos de como isso funcionaria:

| estação GHCND:USC00280907 | estação ? | resultado do `combine_first()` |
| :---: | :---: | :---: |
| 1 | 17 | 1 |
| 1 | `NaN` | 1 |
| `NaN` | 17 | 17 |
| `NaN` | `NaN` | `NaN` |

Veja a 4ª linha - temos `WESF` no lugar correto graças ao índice

In [27]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


### Lidando com valores nulos
Podemos descartar nulos, substituí-los por algum valor arbitrário ou imputá-los usando os dados circundantes. Cada uma dessas opções pode ter ramificações, então devemos escolher com sabedoria.

Podemos usar `dropna()` para descartar linhas onde qualquer coluna tenha um valor nulo. As opções padrão nos deixam com poucos dados:

In [36]:
df_deduped.dropna().shape

(4, 8)

Se passarmos `how='all'`, podemos escolher descartar apenas as linhas onde tudo é nulo, mas isso não remove nada:

In [37]:
df_deduped.dropna(how='all').shape

(324, 8)

Podemos usar apenas um subconjunto de colunas para determinar o que descartar usando o argumento `subset`:

In [38]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

(293, 8)

Isso também pode ser feito ao longo das colunas, e podemos especificar um número mínimo de valores nulos antes de descartar os dados:

In [39]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

Podemos escolher preencher os valores nulos usando `fillna()`:

In [43]:
df_deduped.fillna(value={'WESF': 0}, inplace=True)
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


Neste ponto, fizemos tudo o que podemos sem distorcer os dados. Sabemos que estamos perdendo datas, mas se reindexarmos, não sabemos como preencher os dados `NaN`. Com os dados meteorológicos, não podemos assumir que, porque nevou um dia, irá nevar no próximo, ou que a temperatura será a mesma. Por esse motivo, observe que os próximos exemplos são apenas para fins ilustrativos — só porque podemos fazer algo não significa que devemos.

Dito isso, vamos tentar resolver alguns dos problemas restantes com os dados de temperatura. Sabemos que quando `TMAX` é a temperatura do Sol, é porque não houve valor medido, então vamos substituí-lo por `NaN`. Faremos o mesmo para `TMIN`, que atualmente usa -40°C como espaço reservado, mas sabemos que a temperatura mais fria já registrada em Nova York foi de -15°F (-26,1°C) em 9 de fevereiro de 1934:

In [44]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

Também faremos uma suposição de que a temperatura não mudará drasticamente de um dia para o outro. Observe que esta é uma grande suposição, mas isso nos permitirá entender como funciona o método `fillna()` quando fornecemos uma estratégia através do parâmetro `method`. O método `fillna()` nos dá 2 opções para o parâmetro `method`:

- `'ffill'` para preenchimento para frente
- `'bfill'` para preenchimento para trás

*A opção `'nearest'` está ausente porque não estamos reindexando.*

Aqui, usaremos `'ffill'` para mostrar como isso funciona:

In [45]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill')
).head()

/var/folders/14/lp737qrd6mng398vsjx9zwqw0000gn/T/ipykernel_83172/2746109009.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  TMAX=lambda x: x.TMAX.fillna(method='ffill'),
/var/folders/14/lp737qrd6mng398vsjx9zwqw0000gn/T/ipykernel_83172/2746109009.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  TMIN=lambda x: x.TMIN.fillna(method='ffill')


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


Podemos usar `np.nan_to_num()` para transformar `np.nan` em 0 e `-np.inf`/`np.inf` em números finitos negativos ou positivos grandes:

In [46]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


Dependendo dos dados com os quais estamos trabalhando, podemos usar o método `clip()` como alternativa ao `np.nan_to_num()`. O método `clip()` possibilita limitar os valores a um limite mínimo e/ou máximo específico. Como `SNWD` não pode ser negativo, vamos usar o `clip()` para impor um limite mínimo de zero. Para mostrar como funciona o limite superior, vamos usar o valor de `SNOW`:

In [47]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


Podemos combinar `fillna()` com outros tipos de cálculos. Aqui substituímos os valores ausentes de `TMAX` pela mediana de todos os valores de `TMAX`, `TMIN` pela mediana de todos os valores de `TMIN` e `TOBS` pela média dos valores de `TMAX` e `TMIN`. Como colocamos `TOBS` por último, temos acesso aos valores imputados de `TMIN` e `TMAX` no cálculo:

In [48]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


Também podemos usar `apply()` para executar o mesmo cálculo em todas as colunas. Por exemplo, vamos preencher todos os valores ausentes com a mediana móvel de 7 dias de seus valores, configurando o número de períodos necessários para o cálculo como 0 para garantir que não introduzimos mais valores `NaN` extras. Os cálculos móveis serão abordados no capítulo 4, então este é um preview:

In [49]:
df_deduped.apply(
    # rolling calculations will be covered in chapter 4, this is a rolling 7-day median
    # we set min_periods (# of periods required for calculation) to 0 so we always get a result
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


A última estratégia que podemos tentar é a interpolação com o método `interpolate()`. Especificamos o parâmetro `method` com a estratégia de interpolação a ser usada. Existem muitas opções, mas vamos ficar com o padrão `'linear'`, que tratará os valores como espaçados uniformemente e colocará os valores ausentes no meio dos valores existentes. Como temos alguns dados ausentes, vamos reindexar primeiro. Veja o dia 9 de janeiro, que não tínhamos antes - os valores de `TMAX`, `TMIN` e `TOBS` são a média dos valores do dia anterior (8 de janeiro) e do dia seguinte (10 de janeiro):

In [50]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

/var/folders/14/lp737qrd6mng398vsjx9zwqw0000gn/T/ipykernel_83172/1910715606.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda x: x.interpolate())\


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False


<hr>
<div>
    <a href="./4-reshaping_data.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="../ch_04/1-querying_and_merging.ipynb">
        <button style="float: right;">Chapter 4 &#8594;</button>
    </a>
</div>
<br>
<hr>